# Agent for Event Booking app that guides users through the process of finding and booking tickets for a event. Once they have booked the event they will receive a complimentary custom image related to the event they have booked.

Utilizes
- Sqlite DB
- Tools and using multiple tool functions
- Frontier Model Image Generation
- Frontier Model Audio Generation
- Handling of nested tool calls 
- Gradio sound and image interfaces

In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI
import gradio as gr
import sqlite3
import base64
from io import BytesIO
from PIL import Image

In [ ]:
# initialize api

load_dotenv(override=True)

openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print(f"OpenAI API Key exists")
else:
    print("OpenAI API Key not set")

MODEL = "gpt-4.1-mini"
openai = OpenAI()

In [ ]:
# define sqlite DB

DB = "events.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS events (name TEXT PRIMARY KEY, city TEXT, price REAL, date TEXT, sold_out BOOLEAN)')
    conn.commit()

# Define tool functions

In [ ]:
def get_event_price(name):
    print(f"DATABASE TOOL CALLED: Getting price for {name}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price from events WHERE name = ?', (name.lower(),))
        result = cursor.fetchone()
        return f"Event price to {name} is ${result[0]}" if result else "No price data available for this event try requesting it be added"

In [ ]:
def get_event_date(name):
    print(f"DATABASE TOOL CALLED: Getting date for {name}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT date from events WHERE name = ?', (name.lower(),))
        result = cursor.fetchone()
        return f"Date of event {name} is ${result[0]}" if result else "No date data available for this event try requesting it be added"

In [ ]:
def get_event_city(name):
    print(f"DATABASE TOOL CALLED: Getting city of {name}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT city from events WHERE name = ?', (name.lower(),))
        result = cursor.fetchone()
        return f"Event location is {name} is ${result[0]}" if result else "No location data available for this event try requesting it be added"

In [ ]:
def get_event_capacity(name):
    print(f"DATABASE TOOL CALLED: Getting capacity of {name}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT sold_out from events WHERE name = ?', (name.lower(),))
        result = cursor.fetchone()
        return f"Event {name} is {'Sold Out' if result[0] else 'Available'}" if result else "No availability data available for this event try requesting it be added"

In [ ]:
def book_event(name):
    print(f"DATABASE TOOL CALLED: Booking Ticket {name}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT sold_out from events WHERE name = ?', (name.lower(),)) # should be a insert function in a separate booking table
        result = cursor.fetchone()
        return f"You are {f'booked for {name}' if result[0] else f'not booked for {name}'}" if result else "No availability data available for this event try requesting it be added"


In [ ]:
def set_event(name, city, price, date, sold_out):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute(
            'INSERT INTO events (name, city, price, date, sold_out) VALUES (?, ?, ?, ?, ?) ON CONFLICT(name) DO UPDATE SET city = ?, price = ?, date = ?, sold_out = ?',
            (name.lower(), city, price, date, sold_out, city, price, date, sold_out)
        )
        conn.commit()

In [ ]:
events = [
    {"name": "Lakers-vs-Knicks", "city": "New York", "price": 150.0, "date": "2025-02-25", "sold_out": False},
    {"name": "Seahawks-vs-Patriots", "city": "San Francisco", "price": 520.0, "date": "2025-02-09", "sold_out": True},
    {"name": "Taylor Swift", "city": "Tokyo", "price": 300.0, "date": "2025-03-13", "sold_out": False},
]

for event in events:
    set_event(event["name"], event["city"], event["price"], event["date"], event["sold_out"])

In [ ]:
# System Prompt

system_prompt = """
    You are a helpful assistant for a event booking webiste called EventsForYou.
    You are helping the user find, request and book events like sporting events, concerts and musicals
    Give short, courteous answers, no more than 1 sentence.
    Always be accurate. If you don't know the answer, say so.
    """

# Create tool function definitions

In [ ]:
price_function = {
    "name": "get_event_price",
    "description": "Get the price of a event ticket",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The name of the event the customer would like to attend"
            },
        },
        "required": ["name"],
        "additionalProperties": False
    }
}

date_function = {
    "name": "get_event_date",
    "description": "Get the date of an event",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The name of the event the customer would like to attend"
            },
        },
        "required": ["name"],
        "additionalProperties": False
    }
}

city_function = {
    "name": "get_event_city",
    "description": "Get the city/location where an event is held",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The name of the event the customer would like to attend"
            },
        },
        "required": ["name"],
        "additionalProperties": False
    }
}

capacity_function = {
    "name": "get_event_capacity",
    "description": "Check if an event has tickets available or is sold out",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The name of the event the customer would like to attend"
            },
        },
        "required": ["name"],
        "additionalProperties": False
    }
}

set_event_function = {
    "name": "set_event",
    "description": "Add or update an event in the database (name, city, price, date, sold_out)",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The name of the event"
            },
            "city": {
                "type": "string",
                "description": "The city where the event is held"
            },
            "price": {
                "type": "number",
                "description": "The ticket price"
            },
            "date": {
                "type": "string",
                "description": "The date of the event (YYYY-MM-DD)"
            },
            "sold_out": {
                "type": "boolean",
                "description": "Whether the event is sold out"
            },
        },
        "required": ["name", "city", "price", "date", "sold_out"],
        "additionalProperties": False
    }
}

book_event_function = {
    "name": "book_event",
    "description": "Book a ticket for the customer for the specified event",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The name of the event the customer wants to book"
            },
        },
        "required": ["name"],
        "additionalProperties": False
    }
}

tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": date_function},
    {"type": "function", "function": city_function},
    {"type": "function", "function": capacity_function},
    {"type": "function", "function": set_event_function},
    {"type": "function", "function": book_event_function},
]
tools

# Define chat callback functions

In [ ]:
def artist(name):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a event that the user booked",
            size="1024x1024",
            n=1,
            response_format="b64_json",
    )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [ ]:
def talker(message):
    response = openai.audio.speech.create(
      model="gpt-4o-mini-tts",
      voice="onyx",   
      input=message
    )
    return response.content

In [ ]:
# Map tool name (string) to the callable function
TOOL_FUNCTIONS = {
    "get_event_price": get_event_price,
    "get_event_date": get_event_date,
    "get_event_city": get_event_city,
    "get_event_capacity": get_event_capacity,
    "set_event": set_event,
    "book_event": book_event,
}


def handle_tool_calls(message):
    responses = []
    booked_events = []
    for tool_call in message.tool_calls:
        fn_name = tool_call.function.name
        fn = TOOL_FUNCTIONS.get(fn_name)
        if fn is None:
            content = f"Unknown tool: {fn_name}"
        else:
            arguments = json.loads(tool_call.function.arguments)
            content = fn(**arguments)
            if fn_name == "book_event":
                booked_events.append(arguments.get("name"))
        responses.append({
            "role": "tool",
            "content": str(content),
            "tool_call_id": tool_call.id
        })
    return responses, booked_events


In [ ]:
def event_chat(history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_prompt}] + history #+ [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    booked_events = []
    image = None

    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        result =  handle_tool_calls(message)
        responses = result[0]
        booked_events = result[1] if len(result) > 1 else []
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    reply = response.choices[0].message.content
    history += [{"role": "assistant", "content": reply}]

    voice = talker(reply)

    if booked_events :
        image = artist(booked_events[0])

    return history, voice, image

# Create Gradio UI

In [ ]:
def put_message_in_chatbot(message, history):
    return "", history + [{"role": "user", "content": message}]

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500, interactive=False)
    with gr.Row():
        audio_output = gr.Audio(autoplay=True)
    with gr.Row():
        message = gr.Textbox(label="Chat with our AI Assistant:")
    
    message.submit(put_message_in_chatbot, inputs=[message, chatbot], outputs=[message, chatbot]).then(
        event_chat, inputs=chatbot, outputs=[chatbot, audio_output, image_output]
    )

ui.launch(inbrowser=False, auth=("will", "hello"))